# Тема: Бейзлайны и детерминированные алгоритмы item-item

In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.simplefilter('ignore')

In [163]:
data = pd.read_csv('data/retail_train.csv')
data.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [164]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [165]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [166]:
# общая скидка
data_train['disc'] = np.abs(round(data_train['coupon_disc'] + data_train['coupon_match_disc'] + data_train['retail_disc']))
# выручка
data_train['total'] = round(data_train['quantity'] * data_train['sales_value'])
data_train.disc.sort_values().nunique(), data_train.total.sort_values().nunique() 

(71, 17094)

In [167]:
# группировка ид купленных товаров по пользователям за тестовый период
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
# переименование колонок
result.columns=['user_id', 'actual']
# преобразование массивов в списки
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? - товар не попавший в топ-5000  
Зачем он нужен? - что бы не потерять юзера  
Используя этот товар мы смещаем качество рекомендаций.  
В какую сторону? - в большую сторону  
Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре. - товар 999999 удалять нельзя, т.к. возникнет техническая ошибка. 

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [168]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

Сделайте предсказания

In [169]:
%%time
# товары с большей долей выручки
items_weights = data_train.groupby('item_id')['total'].sum().reset_index()
items_weights = items_weights[items_weights['total']>= 10000]
items_weights['weight'] = items_weights['total'] / items_weights['total'].sum()
items_weights.drop('total', axis=1, inplace=True)

result['weighted_random_recom_total'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 237 ms, sys: 8.03 ms, total: 245 ms
Wall time: 243 ms


,user_id,actual,weighted_random_recom_total
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6544236, 6534178, 6533889, 6534166, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6534166, 6544236, 1404121, 6533889]"


In [170]:
%%time
# товары с большей долей скидки
items_weights = data_train.groupby('item_id')['disc'].sum().reset_index()
items_weights = items_weights[items_weights['disc']>= 50]
items_weights['weight'] = items_weights['disc'] / items_weights['disc'].sum()
items_weights.drop('disc', axis=1, inplace=True)

result['weighted_random_recom_disc'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 529 ms, sys: 7.97 ms, total: 537 ms
Wall time: 536 ms


,user_id,actual,weighted_random_recom_total,weighted_random_recom_disc
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6544236, 6534178, 6533889, 6534166, 397896]","[1044078, 1025611, 825170, 1027849, 960942]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6534166, 6544236, 1404121, 6533889]","[857215, 1003188, 7166863, 5569230, 962229]"


In [171]:
%%time
# товары с большей долей продаж по количеству
items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights = items_weights[items_weights['quantity']>= 1000]
items_weights['weight'] = items_weights['quantity'] / items_weights['quantity'].sum()
items_weights.drop('quantity', axis=1, inplace=True)

result['weighted_random_recom_quantity'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 259 ms, sys: 7.99 ms, total: 267 ms
Wall time: 265 ms


,user_id,actual,weighted_random_recom_total,weighted_random_recom_disc,weighted_random_recom_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6544236, 6534178, 6533889, 6534166, 397896]","[1044078, 1025611, 825170, 1027849, 960942]","[6534178, 6534166, 397896, 6533889, 1388206]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6534166, 6544236, 1404121, 6533889]","[857215, 1003188, 7166863, 5569230, 962229]","[6534178, 6534166, 6533889, 1404121, 6544236]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [172]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [173]:
result.head(2)

,user_id,actual,weighted_random_recom_total,weighted_random_recom_disc,weighted_random_recom_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6544236, 6534178, 6533889, 6534166, 397896]","[1044078, 1025611, 825170, 1027849, 960942]","[6534178, 6534166, 397896, 6533889, 1388206]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6534166, 6544236, 1404121, 6533889]","[857215, 1003188, 7166863, 5569230, 962229]","[6534178, 6534166, 6533889, 1404121, 6544236]"


In [174]:
stata = pd.DataFrame(index=['precision@5'])
for i in result.columns[2:]:
    res = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1)
    stata[i] = round(res.mean(), 3)

stata

,weighted_random_recom_total,weighted_random_recom_disc,weighted_random_recom_quantity
precision@5,0.046,0.023,0.048


In [175]:
result_basic = pd.read_csv('predictions/predictions_basic.csv')# загрузка predict с семианара
result_basic.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[28513, 867021, 1763442, 960508, 7431507]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[875108, 15716298, 35082, 920274, 2131153]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [176]:
result_basic['actual'] = result_basic['actual'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

for i in result_basic.columns[2:]:
    result_basic[i] = result_basic[i].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
    res = result_basic.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1)
    stata[i] = round(res.mean(), 3)

stata.T

,precision@5
weighted_random_recom_total,0.046
weighted_random_recom_disc,0.023
weighted_random_recom_quantity,0.048
random_recommendation,0.001
popular_recommendation,0.155
itemitem,0.137
cosine,0.133
tfidf,0.139
own_purchases,0.180


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров - при применении ограничения товаров до топ-5000 точность алгоритма увеличилась в 5 раз. 
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$ - чем меньше соседей, тем точнее алгоритм.
- Попробуйте стратегии ансамблирования изученных алгоритмов 

In [177]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [178]:
%%time

items = data_train.groupby('item_id')['quantity'].sum().reset_index().\
            sort_values('quantity', ascending=False).head(5000).item_id.tolist()

result_basic['random_recommendation_5000'] = result_basic['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_basic.head(1)

CPU times: user 613 ms, sys: 16 ms, total: 629 ms
Wall time: 628 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[28513, 867021, 1763442, 960508, 7431507]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1008074, 982833, 900072, 850679, 1025581]"


In [179]:
for i in result_basic.columns[2:]:
    res = result_basic.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1)
    stata[i] = round(res.mean(), 3)

stata[['random_recommendation', 'random_recommendation_5000']]

,random_recommendation,random_recommendation_5000
precision@5,0.001,0.005


### Задание 4.* Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.